<a href="https://colab.research.google.com/github/v0idengineer/analisis-sentimen-cs2-svm/blob/main/notebooks/cleaning_dataset_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CLEANING DATASET MODEL**

Model ini bertujuan untuk membersihkan database dari tanda baca, angka, dan karakter khusus. Model ini juga dapat mendeteksi ulasan dalam bahasa Inggris dan Indonesia, jika ulasan tidak mengandung salah satu dari bahasa tersebut maka baris ulasan tersebut akan dihapus(penghapusan berlaku untuk nilai kosong pada review). Terakhir model ini akan memberi label "en" dan "id" pada tiap ulasan yang telah dibersihkan berdasarkan bahasa ulsaan tersebut.

**LIBRARY**

In [ ]:
!pip install langdetect
!pip install Sastrawi

In [ ]:
import pandas as pd
import string
from langdetect import detect

MEMBACA DATASET

In [ ]:
# Load dataset
file_path = 'dataset_v2.csv'  # Ganti dengan path dataset Anda
data = pd.read_csv(file_path)

# Pastikan dataset memiliki kolom review
print("Kolom yang tersedia:", data.columns)
print("Contoh data:\n", data.head())

Kolom yang tersedia: Index(['steamid', 'review', 'timestamp_created', 'timestamp_updated',
       'date_created', 'date_updated'],
      dtype='object')
Contoh data:
              steamid                                             review  \
0  76561198315343800                                        game goblog   
1  76561198354022500  gim ini membuat termotivasi saya menjadi mente...   
2  76561198269106200  It's good game. but RIP Aim, everytime i see m...   
3  76561198387291400                                                cs2   
4  76561198840938800  ok valve, i support ur new game. now tf2 updat...   

   timestamp_created  timestamp_updated date_created date_updated  
0         1695887908         1702898857    9/28/2023   12/18/2023  
1         1695894605         1699954964    9/28/2023   11/14/2023  
2         1695906552         1695906552    9/28/2023    9/28/2023  
3         1695942351         1695942351    9/28/2023    9/28/2023  
4         1695943814         1695943814   

**FUNGSI UNTUK MEMBERSIHKAN TEKS DAN DETEKSI BAHASA**

In [ ]:
# Fungsi untuk menghapus tanda baca, angka, dan karakter khusus
def removepunctuation_and_lowercase(text):
    p = set(string.punctuation)
    text = text.lower()  # Mengubah teks menjadi huruf kecil
    words = text.split()  # Memisahkan teks menjadi kata-kata
    cleaned_text = []

    for i in words:
        # Menghapus tanda baca, angka, dan karakter khusus dari setiap kata
        cleaned_word = ''.join([x for x in i if x not in p])
        cleaned_text.append(cleaned_word)

    return " ".join(cleaned_text)  # Menggabungkan kembali kata-kata yang sudah dibersihkan

# Fungsi untuk mendeteksi bahasa (hanya menerima bahasa Inggris dan Indonesia)
def is_english_or_indonesian(text):
    try:
        lang = detect(text)  # Deteksi bahasa teks
        return lang in ['en', 'id']  # Memeriksa apakah bahasa Inggris ('en') atau Indonesia ('id')
    except:
        return False

In [ ]:
# Membaca dataset
file_path = 'dataset_v2.csv'  # Ganti sesuai path file
data = pd.read_csv(file_path, encoding="ISO-8859-1")

# Menyaring dan membersihkan teks dari kolom tertentu
uncleaned = [i for i in data['review']]  # Ganti 'review' dengan nama kolom yang sesuai
cleaned = []
languages = []  # Menyimpan informasi bahasa

for i in uncleaned:
    try:
        # Memeriksa apakah bahasa teks adalah bahasa Inggris atau Indonesia
        if len(i) == 0 or not is_english_or_indonesian(i):
            raise ValueError("Teks bukan bahasa Inggris atau Indonesia")

        # Menghapus tanda baca, mengubah ke lowercase
        cleaned_text = removepunctuation_and_lowercase(i)

        # Menambahkan bahasa yang terdeteksi
        lang = detect(cleaned_text)  # Deteksi bahasa
        languages.append(lang)  # Menyimpan bahasa

        cleaned.append(cleaned_text)
    except:
        cleaned.append("NAN")  # Menandai dengan "NAN" jika ada masalah dengan teks
        languages.append("NAN")  # Menambahkan "NAN" jika terjadi error deteksi bahasa

# Menghapus baris yang memiliki "NAN" pada kolom 'cleaned_reviews' (data yang tidak valid)
data['cleaned_reviews'] = cleaned
data['language'] = languages
data_cleaned = data[data['language'] != "NAN"]  # Menghapus data yang tidak sesuai bahasa


**CLEANING LANJUTAN**

In [ ]:
# Fungsi untuk menghapus karakter non-ASCII dan angka
def remove_non_ascii_and_numbers(text):
    # Menghapus angka dan karakter non-ASCII
    return ''.join([i for i in text if ord(i) < 128 and not i.isdigit()])

# Terapkan fungsi untuk membersihkan karakter non-ASCII dan angka di kolom 'cleaned_reviews'
data_cleaned['cleaned_reviews'] = data_cleaned['cleaned_reviews'].apply(remove_non_ascii_and_numbers)

# Menampilkan beberapa baris pertama setelah pembersihan
data_cleaned[['review', 'cleaned_reviews']].head()

<ipython-input-6-83c7836e15bf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['cleaned_reviews'] = data_cleaned['cleaned_reviews'].apply(remove_non_ascii_and_numbers)


,review,cleaned_reviews
1,gim ini membuat termotivasi saya menjadi mente...,gim ini membuat termotivasi saya menjadi mente...
2,"It's good game. but RIP Aim, everytime i see m...",its good game but rip aim everytime i see my e...
4,"ok valve, i support ur new game. now tf2 updat...",ok valve i support ur new game now tf update when
5,"too much stutering, jitter, and lagg",too much stutering jitter and lagg
11,Too many cheater,too many cheater


**MENYIMPAN DATASET**

In [ ]:
# Menyimpan data yang sudah dibersihkan ke file baru
data_cleaned.to_csv('dataset_v2_cleaned_no_invalid_language.csv', index=False)

print("Proses pembersihan selesai! Data yang tidak valid sudah dihapus.")

Proses pembersihan selesai! Data yang tidak valid sudah dihapus.


#**STOPWORDS DAN TOKENISASI**

Setelah data dibersihkan langkah selanjutnya adalah tokenisasi, memecah kalimat menjadi unit kata yang lebih kecil, bagian ini juga akan mengecek jika ada karakter yang tidak sesuai maka akan dihapus.

In [ ]:
import nltk
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Download stopwords untuk bahasa Inggris dan Indonesia
nltk.download('stopwords')
nltk.download('punkt')

# Membuat stopword list untuk bahasa Indonesia dan Inggris
factory = StopWordRemoverFactory()
indonesia_stopwords = set(factory.get_stop_words())
english_stopwords = set(stopwords.words('english'))

# Gabungkan stopword dari dua bahasa
combined_stopwords = indonesia_stopwords.union(english_stopwords)

# Fungsi untuk menghapus stopwords
def remove_stopwords(tokens):
    return [word for word in tokens if word not in combined_stopwords]

# Fungsi untuk memperbaiki dan menghapus karakter-karakter yang tidak diinginkan
def clean_weird_chars(text):
    # Menghapus karakter yang di-encode dengan tidak benar (seperti â\x80\x93)
    cleaned_text = text.encode('latin1').decode('utf-8', 'ignore')
    # Menghapus angka dan karakter non-alfabet
    return ''.join([char for char in cleaned_text if char.isalpha() or char.isspace()])

# Fungsi untuk tokenisasi
def tokenize(text):
    return nltk.word_tokenize(text)

# Menghapus karakter-karakter yang tidak diinginkan dan angka dari kolom 'cleaned_reviews'
data_cleaned['cleaned_reviews'] = data_cleaned['cleaned_reviews'].apply(clean_weird_chars)

# Melakukan tokenisasi terlebih dahulu
data_cleaned['tokenized_reviews'] = data_cleaned['cleaned_reviews'].apply(tokenize)

# Menghapus stopwords dari token yang sudah di-tokenize
data_cleaned['stopwords_tokenized_reviews'] = data_cleaned['tokenized_reviews'].apply(remove_stopwords)

# Menampilkan beberapa baris pertama setelah pembersihan dan tokenisasi
data_cleaned[['review', 'cleaned_reviews', 'tokenized_reviews', 'stopwords_tokenized_reviews']].head()

# Menyimpan hasil ke file CSV baru
data_cleaned.to_csv('dataset_v2_cleaned_stopword_tokenized.csv', index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-8-c632e297821e>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['cleaned_reviews'] = data_cleaned['cleaned_reviews'].apply(clean_weird_chars)
<ipython-input-8-c632e297821e>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['tokenized_revie

MENYIMPAN DATASET

In [ ]:
import pandas as pd

# Membaca file dataset_v1_cleaned_stopword_tokenized.csv
data_cleaned = pd.read_csv('dataset_v2_cleaned_stopword_tokenized.csv')

# Menggabungkan token kembali menjadi teks pada kolom 'stopwords_tokenized_reviews'
data_cleaned['cleaned_reviewsv2'] = data_cleaned['stopwords_tokenized_reviews'].apply(lambda x: ' '.join(eval(x)))

# Menyimpan hasil ke file baru
data_cleaned.to_csv('dataset_v1_cleaned_stopword_tokenized_v2.csv', index=False)

# Menampilkan beberapa baris pertama untuk memastikan kolom baru telah dibuat
data_cleaned[['stopwords_tokenized_reviews', 'cleaned_reviewsv2']].head()

,stopwords_tokenized_reviews,cleaned_reviewsv2
0,"['gim', 'membuat', 'termotivasi', 'menjadi', '...",gim membuat termotivasi menjadi menteri komuni...
1,"['good', 'game', 'rip', 'aim', 'everytime', 's...",good game rip aim everytime see enemy either i...
2,"['valve', 'support', 'ur', 'new', 'game', 'tf'...",valve support ur new game tf update
3,"['much', 'stutering', 'jitter', 'lagg']",much stutering jitter lagg
4,"['many', 'cheater']",many cheater


# **LEMMATIZATION**

Sebuah proses untuk mengubah kata menjadi bentuk asalnya, contoh: "berlari" menjadi "lari"

In [ ]:
import spacy
import pandas as pd
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize

# Pastikan resource yang dibutuhkan sudah di-download
nltk.download('punkt')
nltk.download('wordnet')

# Muat model spaCy untuk bahasa Inggris
nlp_en = spacy.load('en_core_web_sm')

# Buat stemmer untuk Bahasa Indonesia menggunakan Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi lemmatisasi untuk bahasa Inggris menggunakan spaCy
def lemmatize_english(text):
    doc = nlp_en(text)
    # Menggabungkan lemma setiap token menjadi satu string
    return " ".join([token.lemma_ for token in doc])

# Fungsi "lemmatization" untuk bahasa Indonesia menggunakan Sastrawi
def lemmatize_indonesian(text):
    # Karena Sastrawi merupakan stemmer, kita tokenisasi dulu
    tokens = word_tokenize(text)
    # Terapkan stemmer untuk masing-masing token
    return " ".join([stemmer.stem(token) for token in tokens])

# Fungsi untuk memilih metode lemmatisasi berdasarkan kolom language
def lemmatize_based_on_language(row):
    text = row['cleaned_reviewsv2']
    lang = row['language']
    # Jika teks kosong atau bukan string, kembalikan string kosong
    if not isinstance(text, str) or text.strip() == "":
        return ""
    if lang == "en":
        return lemmatize_english(text)
    elif lang == "id":
        return lemmatize_indonesian(text)
    else:
        # Jika bahasa tidak dikenali, default gunakan lemmatization bahasa Inggris
        return lemmatize_english(text)

# Membaca dataset dari file CSV
data_cleaned = pd.read_csv('dataset_v_cleaned_stopword_tokenized_v2.csv')

# Terapkan fungsi lemmatize_based_on_language ke setiap baris
data_cleaned['lemmatized_reviews'] = data_cleaned.apply(lemmatize_based_on_language, axis=1)

# Simpan hasil ke file baru
data_cleaned.to_csv('dataset_v2_lemmatized_reviews_final.csv', index=False)

print("✅ Proses lemmatasi berdasarkan language selesai! File disimpan sebagai 'dataset_v1_lemmatized_reviews_final.csv'")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✅ Proses lemmatasi berdasarkan language selesai! File disimpan sebagai 'dataset_v1_lemmatized_reviews_final.csv'
